[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fonnesbeck/vi_pydata_virginia_2025/blob/master/notebooks/Intro_Variational_Inference.ipynb)

# Introduction to Variational Bayesian Methods


In Bayesian analysis, the most common strategy for computing posterior quantities is through Markov Chain Monte Carlo (MCMC). Despite recent advances in efficient sampling, MCMC methods still remain computationally intensive for very large datasets. A more scalable alternative to sampling is Variational Inference (VI), which re-frames the problem of computing the posterior distribution as a minimization of the distance between the true posterior and a member of some approximating family. 

In this section, we provide a basic overview of the VI framework as well as practical examples of its implementation using methods implemented in PyMC.

There are a couple of alternatives to evaluating complex posterior distributions without using an MCMC algorithm:

1. Divide and Conquer approaches: See [this paper](https://arxiv.org/abs/1311.4780) or [this paper](https://arxiv.org/abs/1508.05880) for more details

2. Build an approximation to the posterior $p(\theta|X)$ using some other distribution $q(\theta)$

We will focus on the latter option, and try to use a simple distribution to approximate a complex one.

## Distributional Approximations

One approach is to use a distributional approximation, such as the **Laplace (normal) approximation**. In that scenario, we construct a Taylor series of the posterior and threw away the terms of higher than quadratic order. 

**Variational inference** is another distribitional approximation method. Here, rather than a Taylor series approximation, we choose some class of approximating distributions, and choose the member of that class which is **closest to the posterior**.

Thus, we shift from a stochastic sampling approach to approximation to a deterministic approximation that places bounds on the density of interest, then uses opimization to choose from that bounded set.

## Example: Approximating a Student's-t with a Gaussian 
Let's approximate a Student's t distribution with $\nu =3$  with a Gaussian distribution of some mean and variance.

 
> **Student's-t distribution**
> 
> The Student's-t distribution is symmetric and bell-shaped, like the normal distribution, but has heavier tails, meaning it is more prone to producing values that fall far from its mean. The parameter $\nu$ (nu) controls the overdispersion of the distribution - as $\nu$ increases, the distribution approaches a normal distribution.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import stats

In [ ]:
nu = 3
logp = sp.stats.t(nu).logpdf
logq_norm = sp.stats.norm(0, 1).logpdf

In [ ]:
theta = np.linspace(-5.0, 5.0, 1000)
plt.plot(theta, np.exp(logp(theta)), label='p(theta)')
plt.plot(theta, np.exp(logq_norm(theta)), label='q(theta)')
plt.legend();


How do we measure the distance between two distributions? One naive approach would be to build a set of test points and minimize the mean squared error (MSE) between the $\log p(z)$ and $\log q(z)$. 

$$
{\hat \phi} = \underset{\phi}{{\rm arg\,min}} \frac{\sum_{i} q(\theta_{i};\phi)\left[\log q(\theta_{i};\phi) - \log p(\theta_{i})\right]^{2}}{\sum_{i} q(\theta_{i};\phi)}
$$

### Implementing the approximation

Let's weight points that have more probability in the approximation $q(z)$, the idea being that points near the bulk of  $q(\theta)$ are more important to get right. 

How do we pick $\theta$? Well, let's use the known distribution of  $q(\theta;\phi)$  at each step of the optimization to select a grid of points which sample the regions of highest probability density. Then we will optimize the objective function that we defined in terms of MSE. 

In [ ]:
def logq(theta, mu, lnsigma):
    # log-Gaussian parameterized by mean and log(sigma)
    sigma = np.exp(lnsigma)
    return sp.stats.norm(mu, sigma).logpdf(theta)

Now we can specify an objective function, and fit the normal distribition using `scipy.optimize`

In [ ]:
def regression_vi(logp, n, mu_start, lnsigma_start, atol=1e-6):
    """use an optimizer for simple 1D VI"""
    phi_start = np.array([mu_start, lnsigma_start])
    
    # Objective function. Computes sum above on a grid.
    def obj(phi):
        _sigma = np.exp(phi[1])  # get sigma
        
        # This is the grid, factor of 10 is an arbitrary choice.
        z = np.linspace(phi[0] - 10.0*_sigma , phi[0] + 10.0*_sigma, n)

        # Build weights and differences.
        logqz = logq(z, phi[0], phi[1])
        w = np.exp(logqz)
        diff = logqz - logp(z)
        return np.sum(diff * diff * w) / np.sum(w)

    # Run the optimizer.
    opts = {'disp': True, 'maxiter': 5000, 'maxfev': 5000,
            'fatol': atol, 'xatol': 1e-8}
    phi_hat = sp.optimize.minimize(obj, phi_start,
                                      method='Nelder-Mead',
                                      options=opts)
    print(phi_hat)
    return phi_hat['x'], phi_hat

phi_hat, res = regression_vi(logp, 100, 100.0, -100.0)

In [ ]:
ptheta = np.exp(logp(theta))
qtheta_mse = np.exp(logq(theta, phi_hat[0], phi_hat[1]))

plt.plot(theta, ptheta, label='p(theta)')
plt.plot(theta, qtheta_mse, label='q(theta)')
plt.xlabel('theta')
plt.ylabel('PDF')
plt.legend();

MSE is not an effective criterion for comparing two arbitrary distributions (its only optimal when the approximation error is normally distributed). Instead, we can adopt a more robust information-theoretic measure, such as the **Kullback-Leibler divergence**, which estimates the information distance between two densities.

> **Kullback-Leibler Divergence**
>
> The Kullback-Leibler (KL) divergence is a measure of how one probability distribution differs from another. It represents the relative entropy between two distributions and is always non-negative, being zero only when the distributions are identical. While not a true metric (it's asymmetric), KL divergence is widely used in machine learning for comparing probability distributions.

The objective is to minimize the KL divergence between the approximating distribution and the truth:

$$
\begin{align*}
D_{\rm KL}\big(Q||P\big) &= \int q(\theta) \log\frac{q(\theta)}{p(\theta)}d\theta \\[1.5em]
&= \underbrace{\int q(\theta)\log q(\theta)d\theta}_{\text{entropy of }q(\theta)} - \underbrace{\int q(\theta)\log p(\theta)d\theta}_{\text{expected log likelihood under }q(\theta)}
\end{align*}
$$

In [ ]:
def kl_vi(logp, n, mu_start, lnsigma_start):
    """vi with KL divergence"""
    phi_start = np.array([mu_start, lnsigma_start])
    
    # Objective function. Computes the KL div of q and p.
    def obj(phi):
        # This term is -\int q*log(q).
        # Also known as the differential entropy.
        # For a Gaussian, it can be computed exactly. 
        # See wikipedia or something.
        entropy = phi[1] + 0.5*np.log(2.0 * np.pi) + 0.5 ## 
        
        # now we need to evaluate the second integral in the KL divergence, let's numerically approximate it with a sum
        # This is the grid, factor of 20 is a random choice.
        _sigma = np.exp(phi[1])  # get sigma        
        θ = np.linspace(phi[0] - 20.0*_sigma , phi[0] + 20.0*_sigma, n) #number of grid points 
        dθ = θ[1] - θ[0]  # factor needed for numerical integral
        
        # This term is \int q*log(p)
        logqθ = logq(θ, phi[0], phi[1]) #just evaluate the variational density at each z on the grid and 
                                        # at the current value of phi
        qθ = np.exp(logqθ) # back transform
        return -entropy - np.sum(qθ * logp(θ) * dθ) #KL divergence for this phi
    
    # Pass this objective function to a scipy optimizer
    phi_hat = sp.optimize.minimize(obj, phi_start,
                                      method='Nelder-Mead',
                                      options={'disp': True})
    print(phi_hat)
    return phi_hat['x'], phi_hat

phi_hat, res = kl_vi(logp, 10000, 1.0, 0.0)

Let's have a look at the results:

In [ ]:
theta = np.linspace(-5.0, 5.0, 1000)
ptheta = np.exp(logp(theta))
qtheta_vi = np.exp(logq(theta, phi_hat[0], phi_hat[1]))

plt.plot(theta, ptheta, label='p(theta)')
plt.plot(theta, qtheta_vi, label='q(theta)')
plt.xlabel('theta')
plt.ylabel('PDF')
plt.legend();

Which is the same result as MSE gave us. So, this is as close as we can get with a Gaussian approximation. 

## Exercise: Manual VI

To get a better sense of what is going on here, use the widget below to approximate various target distributions using just a Gaussian variational approximation. Manipulate the parameters of the Gaussian and try to get the best fit for the target distribution, using the KL divergence as the objective.

In [ ]:
from vi_visualization import JupyterVariationalInferenceViz

viz = JupyterVariationalInferenceViz()
viz.show();

## Let's look at a more difficult pdf to approximate:
$$
 \log p(\theta) = 10^{3}\log \theta + \log(1-\theta) - c
$$

where the constant $c \sim Beta(10^3+1, 2)$.

This is an unusual distribution, constrained to the interval $[0,1]$ and with a very sharp peak near 1.

In [ ]:
def logp_hard(theta, a=1e3, b=1):
    val = np.zeros_like(theta)
    msk = (theta >= 1.0) | (theta <= 0.0)
    val[msk] = -np.inf
    if np.any(~msk):
        val[~msk] = a * np.log(theta) + b * np.log(1.0 - theta) - sp.special.betaln(a + 1.0, b + 1.0)
    return val

theta = np.linspace(0.98, 0.999999, 100000)
ptheta = np.exp(logp_hard(theta))
plt.plot(theta, ptheta)
plt.xlabel('theta')
plt.ylabel('PDF');

For convenience, let's visualise this with the logit transformation of $\theta$, which maps $[0,1] \to R$:

$$T(\theta)=\zeta = {\rm logit}(\theta) = \log\left(\frac{\theta}{1-\theta}\right)$$

In [ ]:
theta = np.linspace(0.5, 0.999999, 100000)
ptheta = np.exp(logp_hard(theta))
dtheta_dlogittheta = theta * (1.0 - theta)

plt.plot(sp.special.logit(theta), ptheta * dtheta_dlogittheta, label='p(logit(theta))')
plt.xlabel('logit(theta)')
plt.ylabel('PDF')

A naive approach here would be to proceed as above, fitting a rescaled Gaussian to match the support of $\theta$:

In [ ]:
def logq_unit(theta, mu, lnsigma):
    """log of Gaussian parameterized by mean and log(sigma)
    has unit integral over 0,1 
    and value zero outside of 0,1
    """
    val = np.zeros_like(theta)
    msk = (theta >= 1.0) | (theta <= 0.0)
    val[msk] = -np.inf
    if np.any(~msk):
        sigma = np.exp(lnsigma)
        a, b = (0.0 - mu) / sigma, (1.0 - mu) / sigma
        val[~msk] = sp.stats.truncnorm.logpdf(theta[~msk], a=a, b=b, loc=mu, scale=sigma)
    
    return val

def kl_vi_unit(logp, n, mu_start, lnsigma_start, eps=1e-8):
    """vi with KL divergence over unit integral"""
    phi_start = np.array([mu_start, lnsigma_start])
    
    # Objective function. Computes the KL div of q and p.
    def obj(phi):
        # This term is -\int q*log(q).
        sigma = np.exp(phi[1])
        a, b = (0.0 - phi[0]) / sigma, (1.0 - phi[0]) / sigma
        entropy = sp.stats.truncnorm.entropy(a=a, b=b, loc=phi[0], scale=sigma)

        # This is the grid, factor of 20 is a random choice.
        theta = np.linspace(eps, 1.0 - eps, n)
        dtheta= theta[1] - theta[0]  # factor needed for numerical integral
        
        # This term is \int q*log(p)
        logqtheta = logq_unit(theta, phi[0], phi[1])
        qtheta = np.exp(logqtheta)

        return -entropy - np.sum(qtheta * logp(theta) * dtheta)

    # Run the optimizer.
    phi_hat = sp.optimize.minimize(obj, phi_start,
                                      method='Nelder-Mead',
                                      options={'disp': True, 'maxfev': 10000})
    print(phi_hat)
    return phi_hat['x'], phi_hat

phi_hat, res = kl_vi_unit(logp_hard, 10000, 0.0, 0.0)

In [ ]:
qtheta = np.exp(logq_unit(theta, phi_hat[0], phi_hat[1]))

plt.plot(sp.special.logit(theta), ptheta * dtheta_dlogittheta, label='p(logit(theta))')
plt.plot(sp.special.logit(theta), qtheta * dtheta_dlogittheta, label='q(logit(theta))')
plt.xlabel('logit(theta)')
plt.ylabel('PDF')
plt.legend();

This looks a little weird and rescaling a gaussian to squeeze into the support of $\theta$ is not really a good solution if your parameters all have different supports.

Instead, let's do the reverse and transform the **parameter space** to that of a Gaussian. Again, we can use the logit function.
 
$$
{\rm logit}^{-1}(\zeta) = {\rm sigmoid}(\zeta) = \frac{1}{1 + \exp\left(-\zeta\right)}\\
\frac{d{\rm sigmoid}(\zeta)}{d\zeta} = {\rm sigmoid}(\zeta) \left[1 - {\rm sigmoid}(\zeta)\right]\\
$$
and then our pdf in terms of the transformed parameter is given by
$$
\begin{align*}
 \log p_{\zeta}(\zeta) &= \log p({\rm sigmoid}(\zeta)) + \log\left|\frac{d{\rm sigmoid}(\zeta)}{d\zeta}\right|\\
&= \log p({\rm sigmoid}(\zeta)) + \log {\rm sigmoid}(\zeta) + \log(1-{\rm sigmoid}(\zeta))\ .
\end{align*}
$$

In [ ]:
def logp_easy(logitθ, a=1e3, b=1):
    logabsjac = -1.0 * (np.log(1.0 + np.exp(-logitθ)) + np.log(1.0 + np.exp(logitθ)))
    return (-a * np.log(1.0 + np.exp(-logitθ)) - b * np.log(1.0 + np.exp(logitθ)) + 
            logabsjac - 
            sp.special.betaln(a + 1.0, b + 1.0))

phi_hat, res = kl_vi(logp_easy, 10000, 1.0, 0.0)

In [ ]:
logit_theta = np.linspace(0.0, 14.0, 100000)
plogit_theta = np.exp(logp_easy(logit_theta))
qlogit_theta = np.exp(logq(logit_theta, phi_hat[0], phi_hat[1]))

plt.plot(logit_theta, plogit_theta, label='p(logit(theta))')
plt.plot(logit_theta, qlogit_theta, label='q(logit(theta))')
plt.xlabel('logit(z)')
plt.ylabel('PDF')
plt.legend();

Clearly re-casting $\theta$ to have the same support as a Gaussian makes for nice and easy minimization and fits much better than the other way around. 

We will use this as a general strategy for automating variational inference.


## Approximating an unknown posterior distribution

One obvious limitation of each of the methods we have seen so far is that they all assume that we know the form of the posterior distribution. This is rarely the case in practice.

**What can we do if we don't know the form of the posterior?**

Let's revisit the KL divergence and see what is available to us:

\begin{equation}
\phi^*=\arg\min_{\phi\in\Phi}D_{\rm KL}(q(\theta; \phi) || p(\theta|X))
\end{equation}


The KL divergence is given by

\begin{align*}
D_{\rm KL}\big(Q||P\big) &= \int q(\theta) \log\frac{q(\theta)}{p(\theta|x)}d\theta
\end{align*}

This is the definition of the KL divergence between our approximating distribution $q(\theta)$ and the true posterior $p(\theta|x)$.

We can rewrite this as an expectation with respect to $q$:

\begin{align*}
D_{\rm KL}\big(Q||P\big) &= E_q\left[\log\frac{q(\theta)}{p(\theta|x)}\right]
\end{align*}

Using the properties of logarithms, we can split this into two separate expectations:

\begin{align*}
D_{\rm KL}\big(Q||P\big) &= E_q[\log q(\theta)] - E_q[\log p(\theta|x)]
\end{align*}

Then, applying the definition of conditional probability to the second term, we get:

\begin{align*}
D_{\rm KL}\big(Q||P\big) &= E_q[\log q(\theta)] - E_q[\log \frac{p(\theta,x)}{p(x)}] 
\end{align*}

Finally, let's rearrange the terms:

\begin{align*}
D_{\rm KL}\big(Q||P\big) &= E_q[\log q(\theta)] - E_q[\log p(\theta,x)] + E_q[\log p(x)] \\
&= E_q[\log q(\theta)] - E_q[\log p(\theta,x)] + \log p(x)
\end{align*}

The last step follows because $\log p(x)$ is constant with respect to $\theta$, so $E_q[\log p(x)] = \log p(x)$.

This gives us a quantity called the **Evidence Lower Bound (ELBO)** which is defined as $ELBO = E_q[\log p(\theta,x)] - E_q[\log q(\theta)]$, so:

\begin{align*}
D_{\rm KL}\big(Q||P\big) &= -ELBO + \log p(x)
\end{align*}

Notice that the KL divergence is given by the sum of the ELBO and Model Evidence. To see why, let's revisit the model evidence. 

\begin{align*}
\log p(x)&=&\log\int p(x,\theta)d\theta\\
&=&\log\int p(x,\theta)\frac{q(\theta)}{q(\theta)}d\theta\\
&=&\log(E_{q}\left[\frac{p(x,\theta)}{q(\theta)}\right])\\
&\geq& E_q[\log p(x,\theta)]-E_q[\log q(\theta)].
\end{align*}

This is due to the fact that logarithms are strictly concave, allowing us to invoke [**Jensen's inequality**](http://mathworld.wolfram.com/JensensInequality.html):

> Let $f$ be a convex function (*i.e.* $f^{\prime\prime} \ge 0$) of a random variable X. Then:
> $f(E[X]) \ge E[f(X)]$
>
> And when $f$ is *strictly* convex, then:
> $$E[f(X)] = f(E[X]) \iff X = E[X]$$
> with probability 1.

What this implies for a computational solution is that minimizing the KL divergence is accomplished by maximizing the evidence lower bound.



## Automatic Differentiation Variational Inference: A Tale of Two Transformations

[Kucukelbir et. al. 2015](https://arxiv.org/abs/1603.00788) developed Automatic Differentiation Variational Inference (ADVI) to make variational inference more automated and broadly applicable. The key insight is to transform the inference problem through two clever changes of variables:

1. First, we start with our probabilistic model $p(x,\theta)$ where $\theta$ are the model parameters that may have constraints (e.g. must be positive). 

2. The first transformation $T$ maps the constrained parameters $\theta$ to unconstrained real-valued variables $\zeta = T(\theta)$. For example:
   - Positive parameters are log-transformed
   - Probability simplexes use stick-breaking transforms
   - Bounded intervals use sigmoid transforms
   
   This gives us a new joint distribution $p(x,\zeta)$ where all variables live in $\mathbb{R}^K$.

3. We can now use a simple mean-field Gaussian variational family $q(\zeta;\phi)=N(\zeta|\mu,\text{diag}(\omega^2))$ where $\phi=(\mu,\omega)$ are the variational parameters we need to optimize.

4. The second transformation $S_{\mu,\omega}$ maps a standard normal $\eta \sim N(0,I)$ to our variational distribution via $\zeta = S_{\mu,\omega}(\eta) = \mu + \omega \odot \eta$. This standardization makes Monte Carlo estimation of the ELBO much more efficient.

5. Finally, we can optimize the variational parameters $\phi$ using stochastic gradient ascent on the ELBO:

   $$\phi^* = \arg\max_{\phi} \text{ELBO}(\phi)$$

The figure below illustrates these transformations:
- (a) shows the original constrained parameter space
- (b) shows the unconstrained space after applying $T$ 
- (c) shows the standardized space where we perform efficient Monte Carlo integration

The beauty of ADVI is that these transformations happen automatically - the user only needs to specify their probabilistic model and ADVI handles the rest!


![](images/advi.png)

### Let's get a feel for how this happens 

Remember, we're first going to transform all of our parameters to have the same support in $R^k$. We next need to optimize the KL divergence on the transformed space (like in the sigmoid transformation above). 

To accomplish this, we need to optimize the ELBO for the transformed objective. Our objective function for the transformed variables is now given by the ELBO of the transformation:

$$
ELBO(\phi) = \underbrace{E_{q(\zeta;\phi)}\left[\log p(x,T^{-1}(\zeta))+\log|\det J_{T^{-1}}(\zeta)|\right]}_{\text{Expected log joint density + Jacobian correction}} - \underbrace{E_q[\log q(\zeta;\phi)]}_{\text{Entropy of }q}
$$




This notation is starting to get a litle hairy, so let's return to basics. Remember, the first expectation is just an expectation over a joint likelihood in terms of the approximating variational distribution (hence the correction term).  

This is appoximated by drawing samples from the variational distribution and computing the mean of the log joint density.

$$
\int q(z)\log p(x,T^{-1}(\zeta))|\det J_{T^{-1}}(\zeta)| dz \approx \frac{1}{n}\sum_i \log p(x|\zeta_{i})p(\zeta_{i})
$$

Problem is, we can't differentiate this MC integration with respect to the variational parameters (since they parameterize the distribution from which we draw $\zeta_i$). So we need to transform one more time so that the expectation is in terms of a standard normal (this is called **elliptic standardization**), and them perform MC integration with a draw from a standard normal. 

$$\begin{align}
\int q(\zeta) \log p(x|\zeta)p(\zeta) d\zeta &= \int N(\zeta) \log p(x|\zeta\sigma+\mu)p(\zeta\sigma+\mu) dz \\
&\approx \frac{1}{n}\sum_i \log p(x|\zeta_{i}\sigma+\mu)p(\zeta_{i}\sigma+\mu)\ .
\end{align}$$

Now, since this integral is explicitly in terms of the variational parameters, we can optimize our objective. 

> Above is something we can take derivatives of using a computational backend. We'll use PyMC in a moment. But let's return to our weird pdf.

We can now implement a really crude version of advi on our weird pdf:

In [ ]:
def dlogp_easy_dzeta(zeta):
    theta = sp.special.expit(zeta)
    return (1e3 + 1.0) * (1.0 - theta) - 2 * theta

def kl_vi_with_sgd(dlogp, mu, lnsigma, rng, n_iter=100000, n_draw=1, eta=5e-4, eta_decay=5e-5):
    for i in range(n_iter):
        # Draw the points and convert back to draws from the variational approximation.
        zeta_standard = rng.normal(size=n_draw)
        sigma = np.exp(lnsigma)
        zeta = zeta_standard*sigma + mu
        
        # Compute the derivs.
        dkl_dmu = -np.mean(dlogp(zeta))
        dkl_dlnsigma = -np.mean(dlogp(zeta) * zeta_standard * sigma) - 1
        
        # Now do SGD with the KL divergence.
        lnsigma -= dkl_dlnsigma * eta
        mu -= dkl_dmu * eta
        
        # Decay the learning rate.
        eta *= 1.0 - eta_decay
        
        if not (i % (n_iter // 25)):
            print("iter, mu, lnsigma: % 7d|% 10.4e|% 10.4e" % 
                  (i, mu, lnsigma))
        
    return np.array([mu, lnsigma])

rng = np.random.RandomState(seed=5678)
phi_hat = kl_vi_with_sgd(dlogp_easy_dzeta, 0.0, 0.0, rng)

In [ ]:
logit_theta = np.linspace(0.0, 14.0, 100000)
plogit_theta = np.exp(logp_easy(logit_theta))
qlogit_theta = np.exp(logq(logit_theta, phi_hat[0], phi_hat[1]))

plt.plot(logit_theta, plogit_theta, label='p(logit(theta))')
plt.plot(logit_theta, qlogit_theta, label='q(logit(theta))')
plt.xlabel('logit(theta)')
plt.ylabel('PDF')
plt.legend();

But we certainly don't want to have to compute derivatives all by hand!

## Variational Inference in PyMC

For variational inference specifically, PyMC implements a few algorithms including ADVI.

This automation makes it much easier to apply variational inference to complex models compared to implementing everything manually like we did above.

First, let's briefly cover how to specify a model in PyMC.

### Introduction to PyMC

<img src="images/PyMC.png" width="50%" />

PyMC is a probabilistic programming library for Python that allows users to build and analyze Bayesian statistical models. It provides a flexible framework for specifying models using an intuitive syntax and conducting statistical inference using various state-of-the-art algorithms.

PyMC's feature set helps to make Bayesian analysis as painless as possible. Here is a short list of some of its features:

-   Fits Bayesian statistical models with Markov chain Monte Carlo, variational inference and
    other algorithms.
-   Includes a large suite of well-documented statistical distributions.
-   Leverages `arviz` for convergence diagnostics, model checking methods and creating summaries including tables and plots.
-   Extensible: easily incorporates custom step methods and unusual probability distributions.
-   Non-parametric Bayesian methods, including Gaussian processes and Dirichlet processes, are supported.

PyMC offers intuitive model specification that allows users to define complex statistical models using syntax that closely resembles mathematical notation. The library includes powerful sampling algorithms such as Hamiltonian Monte Carlo (HMC) and the No-U-Turn Sampler (NUTS) for efficient MCMC sampling. For larger datasets, PyMC provides variational inference as a scalable approximation method. The system automatically handles parameter constraints through appropriate transformations, and integrates with ArviZ for comprehensive model diagnostics and visualization.

To get you started, let's see how we can use PyMC to construct a simple logistic regression model.


In [ ]:
np.random.seed(123) 

N_samples = 100
N_features = 2 # Keep it 2D like Figure 1

# True coefficients (e.g., intercept and one slope)
beta_true = np.array([-1.0, 2.0])

# Generate features (e.g., one feature column + intercept column)
X = np.random.randn(N_samples, N_features - 1)
X_design = np.hstack([np.ones((N_samples, 1)), X]) # Add intercept column

# Calculate probabilities
linear_pred = X_design @ beta_true
prob = 1 / (1 + np.exp(-linear_pred))

# Generate binary outcomes
y = stats.bernoulli.rvs(p=prob)

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model')
ax.scatter(X, y, alpha=0.5, label='sampled data')
x_plot = np.linspace(X.min(), X.max(), 100)
x_plot_design = np.hstack([np.ones((100, 1)), x_plot.reshape(-1, 1)])
y_plot = 1 / (1 + np.exp(-x_plot_design @ beta_true))
ax.plot(x_plot, y_plot, 'r-', label='true logistic curve', lw=2)
plt.legend(loc=0)


Logistic regression is a statistical model that predicts binary outcomes by estimating the probability
of an observation belonging to a particular class.

The model takes the form:

$$P(y=1|X) = \sigma(\mu + \beta x) = \frac{1}{1 + e^{-(\mu + \beta x)}}$$

Where:
- $\theta = \mu + \beta x$ is the linear predictor
- $\sigma$ is the sigmoid function that maps any real value to $(0,1)$

The decision boundary is defined by $\mu + \beta x = 0$ where $P(y=1|X) = 0.5$.

In Bayesian inference, we place prior distributions on the parameters $\beta$ and
derive their posterior distribution given the observed data:

$$p(\mu,\beta|X,y) \propto p(y|X, \mu, \beta)p(\mu)p(\beta)$$

### Implementing a PyMC Model

At the model-specification stage (before the data are observed), $y$, $\mu$, and $\beta$ are all random variables. Bayesian "random" variables have not necessarily arisen from a physical random process. The Bayesian interpretation of probability is **epistemic**, meaning random variable $x$'s probability distribution $p(x)$ represents our knowledge and uncertainty about $x$'s value. Candidate values of $x$ for which $p(x)$ is high are relatively more probable, given what we know. 

We can generally divide the variables in a Bayesian model into two types: **stochastic** and **deterministic**. The only deterministic variable in this model is $\theta$. If we knew the values of $\theta$'s parents, we could compute the value of $\theta$ exactly. A deterministic like $\theta$ is defined by a mathematical function that returns its value given values for its parents. Deterministic variables are sometimes called the *systemic* part of the model. The nomenclature is a bit confusing, because these objects usually represent random variables; since the parents of $\theta$ are random, $\theta$ is random also.

On the other hand, even if the values of the parents of variables `y` (before observing the data), $\mu$ or $\beta$ were known, we would still be uncertain of their values. These variables are stochastic, characterized by probability distributions that express how plausible their candidate values are, given values for their parents.

Let's begin by defining the prior distributions for the model parameters. We will use a normal distribution for the intercept $\mu$ and a normal distribution for the slope $\beta$. The prior distributions represent our beliefs about the parameters before observing any data.:

In [ ]:
import pymc as pm

with pm.Model() as model:
    
    # Define priors
    mu = pm.Normal('mu', 0, sigma=5)
    beta = pm.Normal('beta', 0, sigma=5)

We have done two things here. First, we have created a `Model` object; a `Model` is a Python object that encapsulates all of the variables that comprise our theoretical model, keeping them in a single container so that they may be used as a unit. After a `Model` is created, we can populate it with all of the model components that we specified when we wrote the model down. 

Notice that the `Model` above was declared using a `with` statement. This expression is used to define a Python idiom known as a **context manager**. Context managers, in general, are used to manage resources of some kind within a program. In this case, our resource is a `Model`, and we would like to add variables to it so that we can fit our statistical model. The key characteristic of the context manager is that the resources it manages are only defined within the indented block corresponding to the `with` statement. PyMC uses this idiom to automatically add defined variables to a model. Thus, any variable we define is automatically added to the `Model`, without having to explicitly add it.

In fact, PyMC variables cannot be defined without a corresponding `Model`:

In [ ]:
mu = pm.Normal('mu', 0, sigma=5)

PyMC includes most of the common random variable **distributions** used for statistical modeling.

In [ ]:
pm.distributions.__all__

Next we define the variable `theta`, which is a deterministic variable that is defined as a function of the stochastic variables `mu` and `beta`, and the data `x`.

In [ ]:
with model:
    
    theta = mu + beta * X.flatten()


Let's see what we have:

In [ ]:
model.unobserved_RVs

What about `theta`, which is a determinsitic component of the model? In this model, `theta` has not been given a name and given a formal PyMC data structure. It is essentially an **intermediate calculation** in the model, implying that we are not interested in its value when it comes to summarizing the output from the model. Most PyMC objects have a name assigned; these names are used for storage and post-processing:

-   as keys in output databases,
-   as axis labels in plots of traces,
-   as table labels in summary statistics.

If we wish to include `theta` in our output, we need to make it a `Deterministic` object, and give it a name:

```python
theta = pm.Deterministic('theta', mu + beta * x)
```

The last step is to define the **data likelihood**, or sampling distribution. In this case, our measured outcome the binary variable, `y`. This is a stochastic variable but unlike `mu` and `beta` we have *observed* its value. To express this, we set the argument `observed` to the observed outcome data. This tells PyMC that this distribution's value is fixed, and should not be changed:

In [ ]:
with model:
    
    likelihood = pm.Bernoulli('y', p=pm.math.invlogit(theta), observed=y)

In [ ]:
pm.model_to_graphviz(model)

PyMC's `sample` function will fit probability models (linked collections of variables) like ours using Markov chain Monte Carlo (MCMC) sampling. Unless we manually assign particular algorithms to variables in our model, PyMC will assign algorithms that it deems appropriate (it usually does a decent job of this):


In [ ]:
with model:

    trace = pm.sample(1000, tune=2000)

This returns the Markov chain of draws, conventionally called a "trace", from the model in an `InferenceData` object. This is from the `arviz` library that we have been using for plotting, and is an extension of the [xarray](https://xarray.pydata.org/en/stable/) data structure.

In [ ]:
trace

We can visualize the trace using the `plot_trace` function, one of many output processing and visualization functions from `arviz`:

In [ ]:
import arviz as az

az.plot_trace(trace)
plt.tight_layout()

While MCMC is a great way to fit models, we are here to learn about variational inference.

We invoke variational inference with the `fit` function. 

In [ ]:
with model:
    approx = pm.fit(n=20_000)

Analogous to the `InferenceData` object, `approx` contains the variational approximation as an `Approximation` object. 

In [ ]:
approx

We can look at the sequence of loss scores (-ELBO) which are stored in the `hist` attribute.

In [ ]:
plt.plot(approx.hist);

We can now sample from the approximation, to yield a trace in an `InferenceData` object, as we would from MCMC. 

In [ ]:
trace_vi = approx.sample() 
az.plot_trace(trace_vi)
plt.tight_layout();

Here, `trace_vi` acts exactly like a trace from mcmc. 

### Mean field variational inference

By default, `fit` employs a mean field assumption regarding the model parameters. This assumes the variational distribution factors independently: 

$$q(\theta_1, \ldots, \theta_n) = q(\theta_1) \cdots q(\theta_n)$$

We can generalize this to use a full-rank covariance matrix for our approximation, which carries an associated tradeoff in performance.

In [ ]:
with model:
    approx_fr = pm.fit(n=10_000, method='fullrank_advi')

In [ ]:
trace_fullrank = approx_fr.sample() 
az.plot_trace(trace_fullrank)
plt.tight_layout();

Notice that the full-rank approximation is more accurate, but at the cost of increased computation time.

## Example: Mixture model

Let's look at using VI to estimate a simple, but less straightforward model: a mixture model that features a bimodal posterior distribution. Mixture models can be specified easily in PyMC with the `Mixture` class, that accepts a set of densities and an associated set of weights. 

In [ ]:
w = np.array([.2, .8])
mu = np.array([-.3, .5])
sd = np.array([.1, .1])

with pm.Model() as mixture_model:
    x = pm.NormalMixture('x', w=w, mu=mu, sigma=sd)
    x2 = pm.Deterministic('x2', x ** 2)
    sin_x = pm.Deterministic('sin_x', pm.math.sin(x))

In [ ]:
with mixture_model:
    mixture_trace = pm.sample(10_000, tune=5000, target_accept=0.9, cores=2, random_seed=(1, 42))

In [ ]:
az.plot_trace(mixture_trace)
plt.tight_layout()

The NUTS algorithm is easily able to sample from the distribution. Let's try to replicate the analysis with mean-field VI:

In [ ]:
with mixture_model:
    mean_field = pm.fit()

In [ ]:
meanfield_trace = mean_field.sample(500)
az.plot_trace(meanfield_trace, var_names=['x']);

Notice that ADVI has failed to approximate the multimodal distribution, since it uses a Gaussian distribution that has a single mode.

## Checking convergence

The `fit` function supports the use of "callbacks", which are functions that are passed as arguments to other functions, and executed at a specified time within that function. For example, the `CheckParametersConvergence` callback will monitor the model parameter trajectories, and stop the VI algoithm when convergence is achieved.

In [ ]:
from pymc.variational.callbacks import CheckParametersConvergence

with mixture_model:
    mean_field = pm.fit(callbacks=[CheckParametersConvergence()])

In [ ]:
plt.plot(mean_field.hist);

We've reached convergence after a few thousand iterations.

## Tracking parameters

Another useful callback allows users to track parameters directly during the fitting procedure. To do this we need to use the object-oriented (OO) API, which allows direct access to the approximation before inference is performed.

In [ ]:
with mixture_model:
    advi = pm.ADVI()

In [ ]:
advi.approx

Different approximations have different hyperparameters. In mean-field ADVI, we have $\rho$ and $\mu$ (inspired by [Bayes by BackProp](https://arxiv.org/abs/1505.05424)).

In [ ]:
advi.approx.shared_params

There are convenient shortcuts to relevant statistics associated with the approximation. This can be useful, for example, when specifying a mass matrix for NUTS sampling:

In [ ]:
advi.approx.mean.eval(), advi.approx.std.eval()

We can roll these statistics into the `Tracker` callback.

In [ ]:
tracker = pm.callbacks.Tracker(
    mean=advi.approx.mean.eval,  # callable that returns mean
    std=advi.approx.std.eval  # callable that returns std
)

Now, calling `advi.fit` will record the mean and standard deviation of the approximation as it runs.

In [ ]:
approx = advi.fit(15_000, callbacks=[tracker])

We can now plot both the evidence lower bound and parameter traces:

In [ ]:
fig = plt.figure(figsize=(16, 9))
mu_ax = fig.add_subplot(221)
std_ax = fig.add_subplot(222)
hist_ax = fig.add_subplot(212)
mu_ax.plot(tracker['mean'])
mu_ax.set_title('Mean track')
std_ax.plot(tracker['std'])
std_ax.set_title('Std track')
hist_ax.plot(advi.hist)
hist_ax.set_title('Negative ELBO track');

Notice that there are convergence issues with the mean, and that lack of convergence does not seem to change the ELBO trajectory significantly. As we are using the OO API, we can run the approximation longer (using `refine`) until convergence is achieved.

In [ ]:
advi.refine(10_000)

Let's take a look:

In [ ]:
fig = plt.figure(figsize=(16, 9))
mu_ax = fig.add_subplot(221)
std_ax = fig.add_subplot(222)
hist_ax = fig.add_subplot(212)
mu_ax.plot(tracker['mean'])
mu_ax.set_title('Mean track')
std_ax.plot(tracker['std'])
std_ax.set_title('Std track')
hist_ax.plot(advi.hist)
hist_ax.set_title('Negative ELBO track');

We still see evidence for lack of convergence, as the mean has devolved into a random walk. This could be the result of choosing a poor algorithm for inference. At any rate, it is unstable and can produce very different results even using different random seeds.

Let's compare results with the NUTS output:

In [ ]:
import plotly.graph_objects as go

# Get the data
nuts_samples = mixture_trace.posterior['x'].values.ravel()
advi_samples = approx.sample(10_000).posterior['x'].values.ravel()

# Create KDE for both distributions
from scipy import stats
x_range = np.linspace(min(np.min(nuts_samples), np.min(advi_samples)), 
                      max(np.max(nuts_samples), np.max(advi_samples)), 1000)
nuts_kde = stats.gaussian_kde(nuts_samples)
advi_kde = stats.gaussian_kde(advi_samples)

# Create the plotly figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_range, y=nuts_kde(x_range), mode='lines', name='NUTS'))
fig.add_trace(go.Scatter(x=x_range, y=advi_kde(x_range), mode='lines', name='ADVI'))
fig.update_layout(xaxis_title='x', yaxis_title='Density')

So try as we might, ADVI is not able to capture the multimodality of the posterior distribution.

While ADVI is the workhorse algorithm for variational inference in PyMC, there are other algorithms that can often do a better job. Let's look at a couple of them.

## Stein Variational Gradient Descent (SVGD)

SVGD is a general VI algorithm that iteratively shifts a set of particles to match a target posterior. This is done via a form of **functional gradient descent** that minimizes the KL divergence.

1. Sample particles from prior distribution of parameters
2. Formulate bijective (*i.e.* reversible) transformation of particles
3. Iteratively apply transformation until convergence

SVGD directly minimizes $KL(\{x_i\} || p)$ by iteratively moving points $\{x_i\}_{i=1}^n$ towards the target p by updates of form:

$$x_i^{\prime} \leftarrow x_i + \epsilon\phi(x_i)$$

where $\phi$ is a *perturbation direction* chosen to optimally decrease the KL divergence:

$$\phi = \text{argmax}_{\phi}\left\{-\frac{\partial}{\partial \epsilon} KL(q_{[\epsilon \phi]} || p)|_{\epsilon=0}\right\}$$

where $q_{[\epsilon \phi]}$ is the density of $x^{\prime}$ when the density of $x$ is $q$.

The name of the procedure reflects the fact that this direction is related to the **Stein operator**:

$$-\frac{\partial}{\partial \epsilon} KL(q_{[\epsilon \phi]} || p)|_{\epsilon=0} = E_{x \sim q}[\mathcal{T}_p \phi(x)]$$

This animation shows the SVGD algorithm in action:

In [ ]:
from svgd_animated import create_svgd_demo

fig = create_svgd_demo(target='banana', n_particles=50, n_frames=150)
fig.show()

In PyMC, we can invoke SVGD by passing the `method='svgd'` argument to the `fit` function. Method-specific arguments can be passed via the `inf_kwargs` argument. 

For the mixture model, let's set the number of particles to 1000 and use a learning rate of 0.01.

In [ ]:
with mixture_model:
    svgd_approx = pm.fit(300, method='svgd', inf_kwargs=dict(n_particles=1000), 
                         obj_optimizer=pm.sgd(learning_rate=0.01))

In [ ]:
nuts_samples = mixture_trace.posterior['x'].values.ravel()
svgd_samples = svgd_approx.sample(2000).posterior['x'].values.squeeze()

x_range = np.linspace(min(np.min(nuts_samples), np.min(svgd_samples)), 
                      max(np.max(nuts_samples), np.max(svgd_samples)), 1000)
nuts_kde = stats.gaussian_kde(nuts_samples)
svgd_kde = stats.gaussian_kde(svgd_samples)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_range, y=nuts_kde(x_range), mode='lines', name='NUTS'))
fig.add_trace(go.Scatter(x=x_range, y=svgd_kde(x_range), mode='lines', name='SVGD'))
fig.update_layout(xaxis_title='x', yaxis_title='Density')

## Minibatches
When dealing with large datasets, using minibatch training can drastically speed up and improve approximation performance. Large datasets impose a hefty cost on the computation of gradients. 

Minibatch gradient descent is a variation of the gradient descent algorithm that divides a large dataset into (much) smaller batches, each of which are used to calculate gradients, model error and updated model coefficients. These "noisy" gradients are more robust to the presence of local minima, and avoid having to ever use the entire dataset to calculate a gradient.

There is a nice API in PyMC to handle these cases, which is avaliable through the `pm.Minibatch` class. The minibatch is just a highly specialized `TensorVariable`.

To demonstrate, let's simulate a large quantity of data:

In [ ]:
# Raw values
data = np.random.rand(400_000, 100) 
# Scaled values
data *= np.random.randint(1, 5, size=(100,))
# Shifted values
data += np.random.rand(100) * 2   

For comparison, let's fit a model without minibatch processing. This is just a simple model trying to recover the mean and standard deviation of a Gaussian distribution:

In [ ]:
with pm.Model() as model:
    mu = pm.Flat('mu', shape=(100,))
    sd = pm.HalfNormal('sd', shape=(100,))
    like = pm.Normal('like', mu, sd, observed=data)

Just for fun, let's create a custom special purpose callback to halt slow optimization. Here we define a callback that causes a hard stop when approximation runs too slowly:

In [ ]:
def stop_after_10(approx, loss_history, i):
    if (i > 0) and (i % 10) == 0:
        raise StopIteration('I was slow, sorry')

In [ ]:
with model:
    advifit = pm.fit(callbacks=[stop_after_10])

Inference is too slow, taking several seconds per iteration; fitting the approximation would have taken a long time!

Instead, let's use minibatches. At every iteration, we will draw 500 random values. Since PyMC variables are just PyTensor variables, we can swap data in and out of the likelihood at each iteration. However, we need to provide the total size of the dataset in order for the model to be properly specified.

In [ ]:
X = pm.Minibatch(data, batch_size=500)

with pm.Model() as model:
    
    mu = pm.Normal('mu', mu=0., sigma=1.e5, shape=(100,))
    sd = pm.HalfNormal('sd', shape=(100,))
    likelihood = pm.Normal('likelihood', mu, sd, observed=X, total_size=data.size)

In [ ]:
%%time
with model:
    advifit = pm.fit()

In [ ]:
plt.plot(advifit.hist);

Minibatch inference is dramatically faster. Multidimensional minibatches may be needed for some corner cases where you do matrix factorization or model is very wide.

---
## References: 

1. [Variational Inference from the ground up](https://www.civisanalytics.com/blog/variational-inference-ground/)
2. [Automatic Differentiation Variational Inference. Kucukelbir, A., Tran D., Ranganath, R., Gelman, A., and Blei, D. M. (2016)](https://arxiv.org/abs/1603.00788)
3. [Variational Inference: A Review for Statisticians. David M. Blei, Alp Kucukelbir, Jon D. McAuliffe (2016)](https://arxiv.org/abs/1601.00670)

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w